In [1]:
import requests
from bs4 import BeautifulSoup as bs
import re
import json
from wilcoxon.spiderman import *
from wilcoxon.string import *
from wilcoxon.vocab import *
import numpy as np
import chinese
from tqdm import tqdm

In [48]:
getSheets()
chineseV = getVocab('Chinese')

In [36]:
def analyseLyrics(link):

    try:
        site = website(link)
        titleArtist = re.split('( By | Pinyin )',site.html('title')[0].get_text())
        title = titleArtist[0]
        artist = titleArtist[2]

        totalL = ''
        for trans in site.html(style="color: #0000ff;"):
            chineseL = trans.parent.find_previous('p').find_previous('p').get_text()
            translitL = trans.parent.find_previous('p').get_text()
            transL = trans.get_text()
            totalL += chineseL

        if len(totalL) == 0:
            for a in site.html('p',string=re.compile('[\u4e00-\u9fff]+')):
                nextText = a.find_next('p').get_text()
                if bool(re.match('[a-z]',nextText)):
                    totalL += a.get_text()

        totalL = trySimplified(totalL.replace(' ',''))

        analyser = chinese.ChineseAnalyzer()
        totalAnaL = analyser.parse(re.sub('[^\u4e00-\u9fff]+','',totalL))

        chineseV['Importance'] = chineseV['Level'].replace({'HSK 1':0.4,'HSK 2':0.6,'HSK 3':0.8, 'HSK 4':1, 'HSK 5':1, 'HSK 6':0.8, 'Important':1, np.nan:0.5})
        chineseV['Understandability'] = chineseV['Level'].replace({'HSK 1':1,'HSK 2':1,'HSK 3':1, 'HSK 4':0.9, 'HSK 5':0.8, 'HSK 6':0.7, 'Important':0.8, np.nan:0.5})
        chineseDict = chineseV[['Word','Importance','Understandability']].drop_duplicates('Word').set_index('Word').to_dict('index')

        lowhsk = chineseV[chineseV['Understandability'] > 0.8].copy()
        lowhsk['Word'] = lowhsk['Word'].apply(lambda x: list(x))
        lowhsk = lowhsk.explode('Word')[['Word','Importance','Understandability']].drop_duplicates('Word').set_index('Word').to_dict('index')
        words = chineseV['Word'].unique()

        for word in lowhsk:
            if len(re.findall('[\u4e00-\u9fff]+',word)) > 0:
                chineseDict.update({word:lowhsk[word]})

        understandability = 0
        importance = 0
        reinforcedWords = []
        newWords = []

        for token in list(set(totalAnaL.tokens())):
            try:
                understandability += chineseDict[token]['Understandability']
                importance += chineseDict[token]['Importance']
                if chineseDict[token]['Understandability'] < 1 and chineseDict[token]['Importance'] > 0.5:
                    reinforcedWords.append(token)
            except: 
                understandings = []
                for x in token:
                    try: understandings.append(chineseDict[x]['Understandability'])
                    except: understandings.append(0)
                minUnderstanding = min(understandings)
                if minUnderstanding < 0.8: newWords.append(token)

        understandability = int(understandability / len(list(set(totalAnaL.tokens()))) * 100)
        importance = int(importance / len(list(set(totalAnaL.tokens()))) * 100)
        score = importance - understandability
        
        print(score)

        return {'Title':title,
                'Artist':artist,
                'Understandability':understandability,
                'Importance':importance,
                'Score':score,
                'New Words':newWords,
                'Lyrics': totalL,
                'Link':link}
    except: 
        return {'Title':'',
                'Artist':'',
                'Understandability':'',
                'Importance':'',
                'Score':'',
                'New Words':'',
                'Lyrics': '',
                'Link':link}

In [12]:
site = website('https://lyricspinyin.com/')
artists = [a['href'] for a in site.html('ul')[3]('a') if a['href'].count('/') > 5]

In [13]:
allLinks = []
for artist in tqdm(artists):
    site = website(artist)
    for entry in site.html(class_='entry-title'):
        allLinks.append(entry('a')[0]['href'])

100%|██████████| 485/485 [10:12<00:00,  1.26s/it]


In [37]:
results = []
for link in tqdm(allLinks):
    results.append(analyseLyrics(link))

  0%|          | 1/1148 [00:05<1:52:14,  5.87s/it]

-3


  0%|          | 2/1148 [00:09<1:39:20,  5.20s/it]

-4


  0%|          | 3/1148 [00:14<1:35:53,  5.03s/it]

...

-4


100%|█████████▉| 1147/1148 [1:10:12<00:03,  3.57s/it]

-8


100%|██████████| 1148/1148 [1:10:15<00:00,  3.67s/it]

0


In [28]:
analyseLyrics('https://lyricspinyin.com/2020/09/10/ke-le-%e6%b8%b4%e4%ba%86by-%e5%85%ab%e4%b8%89%e5%a4%ad-831-feat-%e6%9f%af%e4%bd%b3%e5%ac%bf-alice-ko-pinyin-lyrics/')

{'Title': 'Ke Le 渴了By 八三夭 831 feat. 柯佳嬿 Alice Ko',
 'Artist': 'Lyrics\xa0 - Lyrics Pinyin',
 'Understandability': 55,
 'Importance': 52,
 'Score': -3,
 'New Words': ['一嗝',
  '负荷',
  '卸',
  '干涸',
  '载',
  '餵',
  '嫌多',
  '浏海',
  '指甲油',
  '紧握着',
  '忐忑',
  '流览'],
 'Link': 'https://lyricspinyin.com/2020/09/10/ke-le-%e6%b8%b4%e4%ba%86by-%e5%85%ab%e4%b8%89%e5%a4%ad-831-feat-%e6%9f%af%e4%bd%b3%e5%ac%bf-alice-ko-pinyin-lyrics/'}

In [39]:
df = pd.DataFrame(results)
df['Score'].replace('',0,inplace=True)
df['Importance'].replace('',0,inplace=True)
df['Understandability'].replace('',0,inplace=True)
df.sort_values('Score',ascending=False).to_clipboard()

In [31]:
results

[{'Title': 'Ke Le 渴了By 八三夭 831 feat. 柯佳嬿 Alice Ko',
  'Artist': 'Lyrics\xa0 - Lyrics Pinyin',
  'Understandability': 55,
  'Importance': 52,
  'Score': -3,
  'New Words': ['一嗝',
   '负荷',
   '卸',
   '干涸',
   '载',
   '餵',
   '嫌多',
   '浏海',
   '指甲油',
   '紧握着',
   '忐忑',
   '流览'],
  'Link': 'https://lyricspinyin.com/2020/09/10/ke-le-%e6%b8%b4%e4%ba%86by-%e5%85%ab%e4%b8%89%e5%a4%ad-831-feat-%e6%9f%af%e4%bd%b3%e5%ac%bf-alice-ko-pinyin-lyrics/'},
 {'Title': 'Po Yun 破雲',
  'Artist': '831 八三夭',
  'Understandability': 53,
  'Importance': 49,
  'Score': -4,
  'New Words': ['荒野',
   '消沉',
   '千疮百孔',
   '牢笼',
   '揭雾',
   '侧身',
   '余晖',
   '追寻',
   '愿承其重',
   '逆光',
   '视死如归',
   '附庸',
   '浮尘',
   '破云',
   '未央',
   '热忱',
   '未',
   '温存'],
  'Link': 'https://lyricspinyin.com/2020/04/29/po-yun-%e7%a0%b4%e9%9b%b2-by-831-%e5%85%ab%e4%b8%89%e5%a4%ad-pinyin-lyrics-and-english-translation/'},
 {'Title': 'You Yi Zhong Bei Shang Jiao Chun Dan 有一種悲傷叫蠢蛋',
  'Artist': '八三夭 A-Lin',
  'Understandability': 61,
  'Im

In [420]:
hhh = website('https://lyricspinyin.com/2020/09/10/ke-le-%e6%b8%b4%e4%ba%86by-%e5%85%ab%e4%b8%89%e5%a4%ad-831-feat-%e6%9f%af%e4%bd%b3%e5%ac%bf-alice-ko-pinyin-lyrics/').site

In [417]:
for a in hhh('p',string=re.compile('[\u4e00-\u9fff]+')):
    nextText = a.find_next('p').get_text()
    if bool(re.match('[a-z]',nextText)):
        print(a)

<p>空調壞了那個午後 衣服濕透貼在胸口</p>
<p>瀏海掩飾忐忑眼神 口紅暈上你的嘴唇</p>
<p>烏雲瞬間散了 就像打了一嗝</p>
<p>愛情渴了 你用你的獨特</p>
<p>愛情渴了 讓我一口乾了</p>
<p>身體渴了 脫掉身上的殼</p>
<p>讓我一口乾了 陪我跳進冰河</p>
<p>快樂不快樂 無關 風格不風格</p>
<p>你的機車不機車 載我流覽世界景色</p>
<p>獨特不獨特 無關 資格不資格</p>
<p>我的 原則不原則 只要抱著你就夠了</p>
<p>懶得想的通通累格 當做說bye-bye的過客</p>
<p>熱情誰會嫌多 再多都能負荷</p>
<p>每條神經都被拉扯 早就不想乖乖坐著</p>
<p>心裡那些垃圾 你總有對策 撤</p>
<p>手指渴了 指甲油都卸了</p>
<p>愛情渴了 還要緊緊握著</p>
<p>身體渴了 脫掉身上的殼</p>
<p>讓我一口乾了 餵飽我的乾涸</p>
<p>快樂不快樂 無關 風格不風格</p>
<p>你的機車不機車 載我流覽世界景色</p>
<p>獨特不獨特 無關 資格不資格</p>
<p>我的 原則不原則 只要抱著你就夠了</p>
<p>渴了 渴了 可不可以渴了</p>
<p>可不可以渴了 可不可以渴了</p>
<p>渴了 渴了 可不可以喝了</p>
<p>可不可以渴了</p>
<p>快樂不快樂 無關風格不風格</p>
<p>你的機車不機車 載我流覽世界景色</p>
<p>獨特不獨特 無關資格不資格</p>
<p>你想唱歌就唱歌 所有噪音都安靜了</p>
<p>快樂不快樂 無關 風格不風格</p>
<p>你的機車不機車 載我流覽世界景色</p>
<p>獨特不獨特 無關 資格不資格</p>
<p>我的 原則不原則 只要抱著你就夠了</p>
<p>渴了 渴了 可不可以渴了</p>
<p>可不可以渴了 可不可以渴了</p>
<p>渴了 渴了 可不可以喝了</p>
<p>可不可以渴了</p>


In [83]:
categories = ['Place','Sports','Health','Name','People','Music','Body','Math','Science','Academic','Life','Food']
clipboard.copy(','.join(chineseV[(chineseV['Purposes'].isin(categories)) & (chineseV['Word'].apply(lambda x: len(x) > 0))]['Word'].apply(tryPinyin).tolist()))


In [52]:
chineseV['Purposes'].unique()

array(['Adjective', 'Adverb', 'Pronoun', 'Verb', 'Noun', 'Conjunction',
       'Preposition', 'Name', 'People', 'Counter', 'Math', 'Grammar',
       'Science', 'Place', 'Life', 'Expression', 'Time', 'Music',
       'Auxiliary', 'Onomatopoeia', 'Body', 'Food', 'Academic', 'Affix',
       'Sports', 'Health', nan, 'Idiom', 'Person', 'Spell out'],
      dtype=object)

In [47]:
import clipboard
clipboard.copy('|'.join(dddd.tolist()))

In [46]:
'|'.join(dddd.tolist())

'大家都|影响|影响|一切|不仅|不得不|不管|不管|不过|不过|不过|专业|专门|专门|世纪|严格|严格|严重|丰富|丰富|主动|主意|举办|举行|乘坐|互相|互联网|交流|交通|京剧|亲戚|仍然|仔细|任何|任务|优秀|伤心|估计|作为|作用|作者|使用|例如|依然|保护|保证|保证|信息|修改|修理|偶尔|儿童|允许|允许|免费|全部|共同|关键|关键|兴奋|其实|其次|其次|养成|内容|冷静|准确|凉快|凉快|减肥|出发|出发|出差|出现|出生|到处|到底|到底|到底|到底|力气|功夫|功夫|动作|勇敢|勺子|千万|博士|占线|印象|危险|即使|压力|厕所|原来|原来|原谅|参观|及时|友好|友谊|反对|发展|发展|发生|另外|另外|只好|可怜|可怜|可怜|可惜|可是|叶子|号码|吃惊|合格|合适|同情|后悔|否则|吸引|周围|味道|咱们|咳嗽|售货员|商量|因此|困难|困难|国籍|国际|地址|地点|地球|坚持|垃圾桶|基础|堵车|塑料袋|填空|复印|复杂|大使馆|大夫|大概|大概|大约|失望|失望|失败|奖金|好像|好处|孙子|安全|安全|安排|安排|安排|安排|安排|安排|安排|安排|安排|安排|完全|完全|客厅|害羞|家具|密码|寒假|对于|对话|对面|对面|导游|将来|尊重|小伙子|尽管|尽管|工资|左右|差不多|差不多|师傅|干杯|平时|年龄|幸福|幽默|幽默|广告|广播|座位|建议|开玩笑|开玩笑|引起|弹钢琴|当时|往往|律师|怀疑|怀疑|态度|性格|恐怕|恐怕|情况|愉快|感动|感情|感觉|成为|成功|房东|打扮|打扮|打扰|打针|批评|批评|技术|护士|报名|抱歉|抽烟|拒绝|按时|按照|排列|排列|排队|接受|接着|接着|提供|提前|提前|提醒|摆脱|支持|支持|收入|收拾|收拾|改变|放弃|放暑假|放松|政府|故意|效果|教授|教授|教育|散步|整理|文章|方向|方法|方面|旅行|无聊|无论|既然|日记|景色|暂时|暖和|暖和|本来|本来|杂志|材料|条件|来不及|来得及|来自|果汁|标准|森林|植物|橡皮|正好|正常|正确|比如|毕业|毛巾|民族|永远|污染|污染|沙发|法律|注意|活动|活动|活动|活泼|流利|流行|浪漫|浪漫|浪漫|浪费|浪费|海洋|消息|激动|激动|烤鸭|烦恼|热闹|热闹|照顾|熟悉|爱情|父亲|牙膏|现金|理发

In [79]:
chineseV[chineseV['Purposes']=='Counter']

,Word,Traditional,Word Pinyin,Purposes,Usages,Examples,Example pinyin,Example translation,Level
63,份,份,fèn,Counter,Measure word for copy of something,哦。那给我来一份吧。,ò。nà gěi wǒ lái yī fèn ba。,"OK. Well, I'll take one order.",HSK 4
64,份,份,fèn,Counter,Measure word for portions share,这是你的第一份工作吗？,zhè shì nǐ de dì yī fèn gōngzuò ma？,Is this your first job?,HSK 4
184,台,颱,tái,Counter,Measure word for certain machinery apparatus a...,您这台是iPhone5是吧？,nín zhè tái shì iPhone 5 shì ba？,"Your cell is an iPhone5, right?",HSK 4
217,场,場,cháng,Counter,Measure word for recreational sports activities,室内滑雪场,shìnèi huáxuěchǎng,indoor ski hill,HSK 4
345,抬,抬,tái,Counter,Measure word for sth that needs 2 people to carry,所以要让你们放松啊，自然一点。好，弹的时候尽量依靠指关节把手指抬高，然后弹下去。对了，很好，...,suǒyǐ yào ràng nǐmen fàngsōng a，zìrán yīdiǎn。h...,"So, I told you all to relax. Be more natural. ...",HSK 4
...,...,...,...,...,...,...,...,...,...
21977,顿,頓,dùn,Counter,measure word for frequency,他被老板骂了一顿。,tā bèi lǎobǎn mà le yídùn。,He got a real telling off by his boys.,NaN
22036,颗,顆,kē,Counter,measure word for round small things,当然咯。你看我手指的方向，那颗就是。,dāngrán lo。nǐ kàn wǒ shǒu zhǐ de fāngxiàng，nà ...,Of course. Look where I'm pointing. That's it...,NaN
22125,首,首,shǒu,Counter,Measure word for songs and poems,这是一首新歌，大家都来听听看。,zhè shì yī shǒu xīn gē，dàjiā dōu lái tīngting ...,"This is a new song. Come and listen, everyone.",NaN
22421,磅,NaN,NaN,Counter,measure word for pound,NaN,NaN,NaN,NaN
